In [ ]:
# default_exp core

# TabNet functionality

> Integrating TabNet with fastai.

In [ ]:
#export
from fastai.basics import *
from fastai.tabular.all import *
from pytorch_tabnet.tab_network import TabNetNoEmbeddings
from pytorch_tabnet.utils import create_explain_matrix
from scipy.sparse import csc_matrix

In [ ]:
#export
class TabNetModel(Module):
    "Attention model for tabular data."
    def __init__(self, emb_szs, n_cont, out_sz, embed_p=0., y_range=None, 
                 n_d=8, n_a=8,
                 n_steps=3, gamma=1.5, 
                 n_independent=2, n_shared=2, epsilon=1e-15,
                 virtual_batch_size=128, momentum=0.02):
        self.emb_szs = emb_szs
        self.embeds = nn.ModuleList([Embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(embed_p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        self.tab_net = TabNetNoEmbeddings(n_emb + n_cont, out_sz, n_d, n_a, n_steps, 
                                          gamma, n_independent, n_shared, epsilon, virtual_batch_size, momentum)
        # embeddings dim reducer for explainability 
        cat_dims = L(emb[1] for emb in emb_szs)
        pre_emb = n_cont + len(emb_szs)
        self.post_emb = n_cont + n_emb
        cat_idxs = list(range(len(emb_szs)))
        self.emb_reducer = create_explain_matrix(pre_emb,
                                            cat_dims,
                                            cat_idxs,
                                            self.post_emb)

    def embedder(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        return x

    def forward(self, x_cat, x_cont):
        x = self.embedder(x_cat, x_cont)
        y, _ = self.tab_net(x)
        if self.y_range is not None:
            y = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(y) + self.y_range[0]
        return y

    def forward_masks(self, x_cat, x_cont):
        x = self.embedder(x_cat, x_cont)
        return self.tab_net.forward_masks(x)

In [ ]:
#export
def tabnet_explain(model:TabNetModel, dl:TabDataLoader):
    "Get explain values for a set of predictions"
    dec_y = []
    model.eval()
    for batch_nb, data in enumerate(dl):
        with torch.no_grad():
            M_explain, masks = model.forward_masks(data[0], data[1])
        for key, value in masks.items():
            masks[key] = csc_matrix.dot(value.cpu().numpy(), model.emb_reducer)

        explain = csc_matrix.dot(M_explain.cpu().numpy(), model.emb_reducer)
        if batch_nb == 0:
            res_explain = explain
            res_masks = masks
        else:
            res_explain = np.vstack([res_explain, explain])                                     
            for key, value in masks.items():
                res_masks[key] = np.vstack([res_masks[key], value])
    return res_explain, res_masks

In [ ]:
#export
def tabnet_feature_importances(model:TabNetModel, dl:TabDataLoader):
    model.eval()
    feature_importances_ = np.zeros((model.post_emb))
    for batch_nb, data in enumerate(dl):
        M_explain, masks = model.forward_masks(data[0], data[1])
        feature_importances_ += M_explain.sum(dim=0).cpu().detach().numpy()

    feature_importances_ = csc_matrix.dot(
        feature_importances_, model.emb_reducer
    )
    return feature_importances_ / np.sum(feature_importances_)

In [ ]:
#export
@patch
def size(x:nn.Module, with_grad: bool=True) -> Int:
    return sum(p.numel() for p in x.parameters() if p.requires_grad or not with_grad)

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script() 

Converted 00_core.ipynb.
Converted 01_examples.ipynb.
Converted index.ipynb.
